In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl

from astropy.table import QTable
from astropy import units as u
from astroquery.jplhorizons import Horizons
from astropy.time import Time
from astropy.timeseries import TimeSeries
  
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['xtick.labelsize'] = 18
mpl.rcParams['ytick.labelsize'] = 18
mpl.rcParams['axes.titlesize'] = 18

%config InlineBackend.figure_format = 'retina'

np.random.seed(42)

In [ ]:
# Load PHEMU occultation lightcurves where Europa occults Io
import os

path = "../data/phemu_catalog/2009/"

lc_occ = []
lc_ec = []
names_occ = ["2o1"]

for file in os.listdir(path):
    if any(x in file for x in names_occ):
        # Grab date
        y, m, d = file[1:5], file[5:7], file[7:9]
        date_mjd = Time(f"{y}-{m}-{d}", format="isot", scale="utc").to_value("mjd")

        lc = np.genfromtxt(os.path.join(path, file))
        times_mjd = date_mjd + lc[:, 0] / (60 * 24)

        timeseries = TimeSeries(time=Time(times_mjd, format="mjd"))
        timeseries["flux"] = lc[:, 1]
        lc_occ.append(timeseries)

In [ ]:
ts = lc_occ[0]

fig, ax = plt.subplots(figsize=(20, 5))
ax.plot(ts.time.datetime, ts["flux"], "ko", alpha=0.5)
ax.grid()
ts.time.iso[0]

In [ ]:
def get_satellite_positions(epochs):
    """
    Returns the positions of Gallilean satellites relative to Jupiter in (RA, DEC) space
    for the selected epochs.
    """
    jup = Horizons(id="599", epochs=epochs, id_type="id")
    io = Horizons(id="501", epochs=epochs, id_type="id")
    europa = Horizons(id="502", epochs=epochs, id_type="id")
    ganymede = Horizons(id="503", epochs=epochs, id_type="id")
    callisto = Horizons(id="504", epochs=epochs, id_type="id")

    # Compute positions of the satellites relative to Jupiter in (RA, DEC) space
    jup_pos = jup.ephemerides(quantities=1, extra_precision=True)
    io_pos = io.ephemerides(quantities=1, extra_precision=True)
    europa_pos = europa.ephemerides(quantities=1, extra_precision=True)
    ganymede_pos = ganymede.ephemerides(quantities=1, extra_precision=True)
    callisto_pos = callisto.ephemerides(quantities=1, extra_precision=True)

    timestamps = jup.ephemerides()["datetime_jd"]

    io_rel_pos = QTable(
        [io_pos["RA"] - jup_pos["RA"], io_pos["DEC"] - jup_pos["DEC"]],
        names=["RA", "DEC"],
    )
    europa_rel_pos = QTable(
        [europa_pos["RA"] - jup_pos["RA"], europa_pos["DEC"] - jup_pos["DEC"]],
        names=["RA", "DEC"],
    )
    ganymede_rel_pos = QTable(
        [ganymede_pos["RA"] - jup_pos["RA"], ganymede_pos["DEC"] - jup_pos["DEC"]],
        names=["RA", "DEC"],
    )
    callisto_rel_pos = QTable(
        [callisto_pos["RA"] - jup_pos["RA"], callisto_pos["DEC"] - jup_pos["DEC"]],
        names=["RA", "DEC"],
    )

    io_rel_pos["RA"].unit = u.deg
    europa_rel_pos["RA"].unit = u.deg
    ganymede_rel_pos["RA"].unit = u.deg
    callisto_rel_pos["RA"].unit = u.deg

    io_rel_pos["DEC"].unit = u.deg
    europa_rel_pos["DEC"].unit = u.deg
    ganymede_rel_pos["DEC"].unit = u.deg
    callisto_rel_pos["DEC"].unit = u.deg

    return [timestamps, io_rel_pos, europa_rel_pos, ganymede_rel_pos, callisto_rel_pos]


def get_satellite_distances(epochs):
    """
    Returns distances of (Jupiter, Io, Europa, Ganymede, Callisto) w.r.t Earth for selected epochs. 
    """
    earth = Horizons(id="399", epochs=epochs, id_type="id")
    jup = Horizons(id="599", epochs=epochs, id_type="id")
    io = Horizons(id="501", epochs=epochs, id_type="id")
    europa = Horizons(id="502", epochs=epochs, id_type="id")
    ganymede = Horizons(id="503", epochs=epochs, id_type="id")
    callisto = Horizons(id="504", epochs=epochs, id_type="id")

    # Cartesian vectors
    earth_vec = earth.vectors()
    jup_vec = jup.vectors()
    io_vec = io.vectors()
    europa_vec = europa.vectors()
    ganymede_vec = ganymede.vectors()
    callisto_vec = callisto.vectors()

    def compute_distance(r1, r2):
        return (
            np.sqrt(
                (r2["x"] - r1["x"]) ** 2
                + (r2["y"] - r1["y"]) ** 2
                + (r2["z"] - r1["z"]) ** 2
            )
            * u.au
        )

    jup_distance = compute_distance(earth_vec, jup_vec)
    io_distance = compute_distance(earth_vec, io_vec)
    europa_distance = compute_distance(earth_vec, europa_vec)
    ganymede_distance = compute_distance(earth_vec, ganymede_vec)
    callisto_distance = compute_distance(earth_vec, callisto_vec)

    return [
        jup_distance,
        io_distance,
        europa_distance,
        ganymede_distance,
        callisto_distance,
    ]


# Observation times
epochs = {"start": "2009-12-04T08:53:00", "stop": "2009-12-04T09:00:00", "step": "1m"}

timestamps, pos_io, pos_europa, pos_ganymede, pos_callisto = get_satellite_positions(
    epochs
)
dist_jup, dist_io, dist_europa, dist_ganymede, dist_callisto = get_satellite_distances(
    epochs
)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

ax.plot(pos_io["RA"], pos_io["DEC"], "C0o", alpha=0.5)
ax.plot(pos_europa["RA"], pos_europa["DEC"], "C1o", alpha=0.5)
ax.plot(pos_ganymede["RA"], pos_ganymede["DEC"], "C2o", alpha=0.5)
ax.plot(pos_callisto["RA"], pos_callisto["DEC"], "C3o", alpha=0.5)
ax.set_xlabel("RA")
ax.set_ylabel("DEC")

ax.grid()

In [ ]:
ang_radius_jup = (69911 * u.km / dist_jup.to(u.km)) * u.rad
ang_radius_io = (1821.3 * u.km / dist_io.to(u.km)) * u.rad
ang_radius_europa = (1565.0 * u.km / dist_europa.to(u.km)) * u.rad
ang_radius_ganymede = (2634.0 * u.km / dist_ganymede.to(u.km)) * u.rad
ang_radius_callisto = (2403.0 * u.km / dist_callisto.to(u.km)) * u.rad


def plot_circles(ax, r1, r2, rad1, rad2):
    c1 = plt.Circle((r1[0], r1[1]), rad1, color="C0")
    c2 = plt.Circle((r2[0], r2[1]), rad2, color="C1")

    ax.add_artist(c1)
    ax.add_artist(c2)

    ax.legend([c1, c2], ["Io", "Europa"])


fig, ax = plt.subplots(figsize=(8, 8))

idx = 3

plot_circles(
    ax,
    [pos_io["RA"][idx].to(u.arcsec).value, pos_io["DEC"][idx].to(u.arcsec).value],
    [
        pos_europa["RA"][idx].to(u.arcsec).value,
        pos_europa["DEC"][idx].to(u.arcsec).value,
    ],
    ang_radius_io[idx].to(u.arcsec).value,
    ang_radius_europa[idx].to(u.arcsec).value,
)

ax.set_ylim(20, 23)
ax.set_xlim(56, 60)
ax.set_aspect(1)
ax.grid()
ax.set_xlabel("x [arcsec]")
ax.set_ylabel("y [arcsec]")
Time(timestamps[idx], format="jd").isot

### Comparison to IMCCE multisat ephemeris

In [ ]:
eph_jup = np.genfromtxt("imcce_calculator_jupiter.txt")
eph_io = np.genfromtxt("imcce_calculator_io.txt")
eph_europa = np.genfromtxt("imcce_calculator_europa.txt")

times = TimeSeries(time=Time(eph_jup[:, 0], format="mjd"))

ra_jup = eph_jup[:, 1] * 360 / 24 * u.deg
dec_jup = eph_jup[:, 2] * u.deg

ra_io = eph_io[:, 1] * 360 / 24 * u.deg
dec_io = eph_io[:, 2] * u.deg

ra_europa = eph_europa[:, 1] * 360 / 24 * u.deg
dec_europa = eph_europa[:, 2] * u.deg

ra_io_rel = ra_io - ra_jup
dec_io_rel = dec_io - dec_jup

ra_europa_rel = ra_europa - ra_jup
dec_europa_rel = dec_europa - dec_jup

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

ax[0].plot(
    times.time.mjd,
    pos_io["RA"].to(u.arcsec).value - ra_io_rel.to(u.arcsec).value,
    "C0o",
    label="Io",
)
ax[0].plot(
    times.time.mjd,
    pos_europa["RA"].to(u.arcsec).value - ra_europa_rel.to(u.arcsec).value,
    "C1o",
    label="Europa",
)
ax[1].plot(
    times.time.mjd,
    pos_io["DEC"].to(u.arcsec).value - dec_io_rel.to(u.arcsec).value,
    "C0o",
    label="Io",
)
ax[1].plot(
    times.time.mjd,
    pos_europa["DEC"].to(u.arcsec).value - dec_europa_rel.to(u.arcsec).value,
    "C1o",
    label="Europa",
)


ax[0].set_ylabel("RA difference [arcsec]")
ax[1].set_ylabel("DEC difference [arcsec]")

ax[1].set_xlabel("Time")

for a in ax.flatten():
    a.grid()
    a.legend()


ax[0].set_title("Difference in ephemeris between JPL Horizons and IMCCE Multisat")

### Illumination of Io as a function of time

In [ ]:
# Observation times
epochs = {"start": "2002-12-04", "stop": "2003-12-04", "step": "6h"}

io = Horizons(id="501", epochs=epochs, id_type="id")
io_eph = io.ephemerides()
io_eph.info

![](http://win98.altervista.org/space/exploration/Q15.png)
This picture is wrong, longitudes are positive Westward not Eastward.

In [ ]:
# Cartesian vectors
earth = Horizons(id="399", epochs=epochs, id_type="id")

earth_vec = earth.vectors()
io_vec = io.vectors()


def compute_distance(r1, r2):
    return (
        np.sqrt(
            (r2["x"] - r1["x"]) ** 2
            + (r2["y"] - r1["y"]) ** 2
            + (r2["z"] - r1["z"]) ** 2
        )
        * u.au
    )


io_distance = compute_distance(earth_vec, io_vec)

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(15, 10), sharex=True)
ax[0].plot(
    io_eph["datetime_jd"] - io_eph["datetime_jd"][0],
    io_eph["PDSunLon"] - io_eph["PDObsLon"],
    "C0o",
)

ax[0].set_yticks([-360, -180, 0, 180, 360])

ax[0].set_ylabel("Lon(Sun) - Lon(Cent)")

ax[1].plot(
    io_eph["datetime_jd"] - io_eph["datetime_jd"][0],
    io_eph["PDSunLat"] - io_eph["PDObsLat"],
    "C0o",
)
# ax[1].set_yticks([0, 90, 180, 270, 360])
ax[1].set_ylabel("Lat(Sun) - Lat(Cent)")

ax[2].plot(
    io_eph["datetime_jd"] - io_eph["datetime_jd"][0],
    io_eph["illum_defect"].to(u.rad) * io_distance.to(u.km),
    "C0o",
)
# ax[1].set_yticks([0, 90, 180, 270, 360])
ax[2].set_ylabel("Illumination defect [km]")

ax[2].set_xlabel(f"Days since {epochs['start']}")

# ax[0].set_ylim(-12, 12)
# ax[0].set_yticks(np.arange(-12, 14,  4))


for a in ax.flatten():
    a.grid()

In [ ]:
import starry

starry.config.lazy = False

map = starry.Map(ydeg=0, reflected=True)

fig, ax = plt.subplots(figsize=(12, 7))
map.show(ax=ax, projection="rect")

In [ ]:
io_cent_lon = io_eph["PDObsLon"].to(u.rad)
io_cent_lat = io_eph["PDObsLat"].to(u.rad)

subsolar_lon = io_eph["PDSunLon"].to(u.rad)
subsolar_lat = io_eph["PDSunLat"].to(u.rad)

# Get Longitude of Sun relative to central point of Io, same for Latitude
subtract_angles = lambda x, y: np.fmod((x - y) + np.pi * 3, 2 * np.pi) - np.pi

subsolar_lon_diff = subtract_angles(np.array(subsolar_lon), np.array(io_cent_lon))
subsolar_lat_diff = subtract_angles(np.array(subsolar_lat), np.array(io_cent_lat))

ig, ax = plt.subplots(3, 1, figsize=(15, 8), sharex=True)

ax[0].plot(
    io_eph["datetime_jd"] - io_eph["datetime_jd"][0], subsolar_lon * 180 / np.pi, "C0o"
)
ax[0].set_ylabel("PDSunLon")
ax[1].plot(
    io_eph["datetime_jd"] - io_eph["datetime_jd"][0], io_cent_lon * 180 / np.pi, "C1o"
)
ax[1].set_ylabel("PDObsLon")
ax[2].plot(
    io_eph["datetime_jd"] - io_eph["datetime_jd"][0],
    subsolar_lon_diff * 180 / np.pi,
    "C2o",
)
ax[2].set_ylabel("Difference")

ax[0].set_yticks([0, 90, 180, 270, 360])
ax[1].set_yticks([0, 90, 180, 270, 360])

for a in ax.ravel():
    a.grid()

In [ ]:
# Convert to cartesian coordinates used in Starry
xs = np.array(io_eph["r"]) * np.cos(subsolar_lat_diff) * np.sin(subsolar_lon_diff)
ys = np.array(io_eph["r"]) * np.sin(subsolar_lat_diff)
zs = np.array(io_eph["r"]) * np.cos(subsolar_lat_diff) * np.cos(subsolar_lon_diff)

fig, ax = plt.subplots(3, 1, figsize=(15, 8), sharex=True)
ax[0].plot(io_eph["datetime_jd"] - io_eph["datetime_jd"][0], xs, "C0o")
ax[1].plot(io_eph["datetime_jd"] - io_eph["datetime_jd"][0], ys, "C1o")
ax[2].plot(io_eph["datetime_jd"] - io_eph["datetime_jd"][0], zs, "C2o")
ax[0].set_ylabel("xs")
ax[1].set_ylabel("ys")
ax[2].set_ylabel("zs")

for a in ax.flatten():
    a.grid()

In [ ]:
import starry

starry.config.lazy = False

map = starry.Map(ydeg=0, reflected=True)

fig, ax = plt.subplots(figsize=(12, 7))
map.show(ax=ax, projection="rect")

fig, ax = plt.subplots(figsize=(8, 8))

map.obl = -20
map.inc = 90 - 20
map.show(ax=ax, xs=xs[::5], ys=ys[::5], zs=zs[::5])

In [ ]:
# Observation times
epochs = {"start": "2002-12-04", "stop": "2005-12-04", "step": "2d"}

io = Horizons(id="501", epochs=epochs, id_type="id")
jup = Horizons(id="599", epochs=epochs, id_type="id")

io_eph = io.ephemerides()
io_eph[["NPole_ang", "NPole_RA", "NPole_DEC"]]
jup_eph = jup.ephemerides()
jup_eph[["NPole_ang", "NPole_RA", "NPole_DEC"]]

In [ ]:
jup_eph = jup.ephemerides()
jup_eph[["NPole_ang", "NPole_RA", "NPole_DEC"]]

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
ax.plot(
    io_eph["datetime_jd"] - io_eph["datetime_jd"][0],
    io_eph["NPole_ang"] - jup_eph["NPole_ang"],
    "C0o",
    label="Io",
    alpha=0.3,
)
# ax.plot(
#     io_eph["datetime_jd"] - io_eph["datetime_jd"][0],
#     jup_eph["NPole_ang"],
#     "C1o",
#     label="Jupiter",
#     alpha=0.3,
# )

ax.grid()
ax.legend()
ax.set_ylabel("Body NP angle CCW")
ax.set_xlabel(f"Days since {epochs['start']}")

In [ ]:
def get_starry_map_orientation(times, body_id="501", step="1m"):
    start = times.isot[0]

    # because Horizons time range doesn't include the endpoint we need to add some extra time
    if step[-1] == "m":
        padding = 2 * float(step[:-1]) / (60 * 24)
    elif step[-1] == "h":
        padding = 2 * float(step[:-1]) / 24
    elif step[-1] == "d":
        padding = 2 * float(step[:-1])
    else:
        raise ValueError(
            "Unrecognized JPL Horizons step size. Use '1m' or '1h' for example."
        )

    end = Time(times.mjd[-1] + padding, format="mjd").isot

    # Query JPL Horizons
    epochs = {"start": start, "stop": end, "step": step}
    obj = Horizons(id=body_id, epochs=epochs, id_type="id")
    eph = obj.ephemerides(quantities="1,11,12,13,14,15,17, 19", extra_precision=True)
    times_jpl = Time(eph["datetime_jd"], format="jd")

    # Store all data in a TimeSeries object
    data = TimeSeries(time=times)

    data["RA"] = np.interp(times.mjd, times_jpl.mjd, eph["RA"]) * eph["RA"].unit
    data["DEC"] = np.interp(times.mjd, times_jpl.mjd, eph["DEC"]) * eph["DEC"].unit
    data["ang_width"] = (
        np.interp(times.mjd, times_jpl.mjd, eph["ang_width"]) * eph["ang_width"].unit
    )
    data["illum_defect"] = (
        np.interp(times.mjd, times_jpl.mjd, eph["illum_defect"])
        * eph["illum_defect"].unit
    )
    data["par_ecl"] = np.array(
        np.interp(times.mjd, times_jpl.mjd, eph["sat_vis"] == "p"), dtype=bool
    )
    data["tot_ecl"] = np.array(
        np.interp(times.mjd, times_jpl.mjd, eph["sat_vis"] == "u"), dtype=bool
    )
    data["occ_primary"] = np.array(
        np.interp(times.mjd, times_jpl.mjd, eph["sat_vis"] == "O"), dtype=bool
    )

    # Helper functions for dealing with angles and discontinuities
    subtract_angles = lambda x, y: np.fmod((x - y) + np.pi * 3, 2 * np.pi) - np.pi

    def interpolate_angle(x, xp, yp):
        """Interpolate an angular quantity on domain [-pi, pi) and avoid discountinuities."""
        cosy = np.interp(x, xp, np.cos(yp))
        siny = np.interp(x, xp, np.sin(yp))

        return np.arctan2(siny, cosy)

    # Inclination of the starry map = 90 - latitude of the central point of the observed disc
    data["inc"] = interpolate_angle(
        times.mjd, times_jpl.mjd, np.pi / 2 * u.rad - eph["PDObsLat"].to(u.rad)
    ).to(u.deg)

    # Rotational phase of the starry map is the observer longitude
    data["theta"] = (
        interpolate_angle(
            times.mjd, times_jpl.mjd, eph["PDObsLon"].to(u.rad) - np.pi * u.rad
        ).to(u.deg)
    ) + 180 * u.deg

    # Obliquity of the starry map is minus the CCW angle from the celestial NP to the NP of the target body
    data["obl"] = interpolate_angle(
        times.mjd, times_jpl.mjd, eph["NPole_ang"].to(u.rad)
    ).to(u.deg)

    data["PDObsLon"] = (
        interpolate_angle(
            times.mjd, times_jpl.mjd, eph["PDObsLon"].to(u.rad) - np.pi * u.rad
        ).to(u.deg)
        + 180 * u.deg
    )

    data["PDSunLon"] = (
        interpolate_angle(
            times.mjd, times_jpl.mjd, eph["PDSunLon"].to(u.rad) - np.pi * u.rad
        ).to(u.deg)
        + 180 * u.deg
    )

    data["PDObsLat"] = (
        interpolate_angle(
            times.mjd, times_jpl.mjd, eph["PDObsLat"].to(u.rad) - np.pi * u.rad
        ).to(u.deg)
        + 180 * u.deg
    )

    data["PDSunLat"] = (
        interpolate_angle(
            times.mjd, times_jpl.mjd, eph["PDSunLat"].to(u.rad) - np.pi * u.rad
        ).to(u.deg)
        + 180 * u.deg
    )

    # Compute the location of the subsolar point relative to the central point of the disc
    lon_subsolar = subtract_angles(
        np.array(eph["PDSunLon"].to(u.rad)), np.array(eph["PDObsLon"].to(u.rad))
    )

    lat_subsolar = subtract_angles(
        np.array(eph["PDSunLat"].to(u.rad)), np.array(eph["PDObsLat"].to(u.rad))
    )

    lon_subsolar = 2 * np.pi - lon_subsolar  # JPL longitude increases to the west

    # Location of the subsolar point in cartesian Starry coordinates
    xs = np.array(eph["r"]) * np.cos(lat_subsolar) * np.sin(lon_subsolar)
    ys = np.array(eph["r"]) * np.sin(lat_subsolar)
    zs = np.array(eph["r"]) * np.cos(lat_subsolar) * np.cos(lon_subsolar)

    data["xs"] = np.interp(times.mjd, times_jpl.mjd, xs) * u.AU
    data["ys"] = np.interp(times.mjd, times_jpl.mjd, ys) * u.AU
    data["zs"] = np.interp(times.mjd, times_jpl.mjd, zs) * u.AU

    return data

In [ ]:
times = Time(np.linspace(ts.time.mjd[0], ts.time.mjd[-1] + 10, 10000), format="mjd")

data = get_starry_map_orientation(times, step="5m")
data

In [ ]:
fig, ax = plt.subplots(4, 1, figsize=(15, 20))
fig.subplots_adjust(hspace=0.1)

ax[0].plot(
    data.time.mjd - data.time.mjd[0],
    data["illum_defect"] / data["ang_width"] * 100,
    "C0o",
)
ax[0].set_ylabel("% of diameter not illuminated")

ax[1].plot(
    data.time.mjd - data.time.mjd[0],
    data["par_ecl"],
    "C0o",
    alpha=0.5,
    label="partial eclipse",
)
ax[1].plot(
    data.time.mjd - data.time.mjd[0],
    data["tot_ecl"],
    "C1o",
    alpha=0.5,
    label="total eclipse",
)
ax[1].plot(
    data.time.mjd - data.time.mjd[0],
    data["occ_primary"],
    "C2o",
    alpha=0.5,
    label="occultation",
)
ax[1].legend()
ax[1].set_ylabel("Eclipse/occultation")
ax[2].plot(data.time.mjd - data.time.mjd[0], data["theta"], "C0o")
ax[2].set_ylabel("Theta [deg]")
ax[3].plot(data.time.mjd - data.time.mjd[0], data["obl"], "C0o")
ax[3].set_ylabel("Obliquity [deg]")

ax[-1].set_xlabel("Days")
for a in ax.flatten():
    a.grid()

In [ ]:
times = Time(
    np.linspace(ts.time.mjd[0] - 10 * 365, ts.time.mjd[-1] + 10 * 365, 10000),
    format="mjd",
)
data = get_starry_map_orientation(times, step="1d")

fig, ax = plt.subplots(4, 1, figsize=(15, 20))
fig.subplots_adjust(hspace=0.1)

ax[0].plot(
    data.time.mjd - data.time.mjd[0],
    data["illum_defect"] / data["ang_width"] * 100,
    "C0o",
)
ax[0].set_ylabel("% of diameter not illuminated")

ax[1].plot(data.time.decimalyear, data["inc"], "C0o")
ax[1].set_ylabel("Inclination [deg]")

ax[2].plot(data.time.decimalyear, data["obl"], "C0o")
ax[2].set_ylabel("Obliquity [deg]")
ax[3].plot(data.time.decimalyear, data["xs"], "C0o", label="xs")
ax[3].plot(data.time.decimalyear, data["ys"], "C1o", label="ys")
ax[3].plot(data.time.decimalyear, data["zs"], "C2o", label="zs")
ax[3].legend()
ax[3].set_ylabel("Source position [AU]")


ax[-1].set_xlabel("Year")
for a in ax.flatten():
    a.grid()

In [ ]:
start = Time("2002-03-28", format="isot")
stop = Time("2002-04-28", format="isot")

epochs = {"start": start.isot, "stop": stop.isot, "step": "1h"}

io = Horizons(id="501", epochs=epochs, id_type="id", location='@sun')
sun = Horizons(id="Sun", epochs=epochs, id_type="id", location='@sun')
earth = Horizons(id="399", epochs=epochs, id_type="id", location='@sun')

io_eph = io.ephemerides(extra_precision=True)

io_vec = io.vectors()
sun_vec = sun.vectors()
earth_vec = earth.vectors()

In [ ]:
idx = 0

fig, ax = plt.subplots(figsize=(8, 6))

ax.scatter(
    np.array(sun_vec["x"][idx]),
    np.array(sun_vec["y"][idx]),
    marker="o",
    color="C1",
    label="Sun",
)
ax.scatter(
    np.array(earth_vec["x"][idx]),
    np.array(earth_vec["y"][idx]),
    marker="o",
    color="C0",
    label="Earth",
)
ax.scatter(
    np.array(io_vec["x"][idx]),
    np.array(io_vec["y"][idx]),
    marker="o",
    color="C2",
    label="Io",
)
ax.legend()

ax.set_xlim(-7, 7)
ax.set_ylim(-7, 7)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.grid()

print("Observer longitude", io_eph["PDObsLon"][idx], "W")
print("Sun longitude", io_eph["PDSunLon"][idx], "W")
print("Difference", io_eph["PDSunLon"][idx] - io_eph["PDObsLon"][idx], "W")
print("Phase angle", io_eph["alpha_true"][idx])

print("Observer latitude", io_eph["PDObsLat"][idx])
print("Sun latitude", io_eph["PDSunLat"][idx])
print("Difference", io_eph["PDSunLat"][idx] - io_eph["PDObsLat"][idx])
print(Time(io_eph["datetime_jd"][idx], format="jd").isot)

In [ ]:
times = Time(np.linspace(start.mjd, stop.mjd, 1000), format="mjd")
data = get_starry_map_orientation(times, step="1h")


idx = 0

fig, ax = plt.subplots(1, 2, figsize=(15, 6))

ax[0].scatter(
    np.array(sun_vec["x"][idx]),
    np.array(sun_vec["y"][idx]),
    marker="o",
    color="C1",
    label="Sun",
)
ax[0].scatter(
    np.array(earth_vec["x"][idx]),
    np.array(earth_vec["y"][idx]),
    marker="o",
    color="C0",
    label="Earth",
)
ax[0].scatter(
    np.array(io_vec["x"][idx]),
    np.array(io_vec["y"][idx]),
    marker="o",
    color="C2",
    label="Io",
)
ax[0].legend()

ax[0].set_xlim(-7, 7)
ax[0].set_ylim(-7, 7)
ax[0].grid()

map = starry.Map(ydeg=30, reflected=True)

# Plot io
map.obl = data["obl"][idx]
map.inc = data["inc"][idx]
map.show(
    ax=ax[1],
    theta=data["theta"][idx],
    xs=data["xs"][idx],
    ys=data["ys"][idx],
    zs=data["zs"][idx],
)
ax[1].axis("on")

print("Observer longitude", data["PDObsLon"][idx], "W")
print("Sun longitude", data["PDSunLon"][idx], "W")
print("Difference", data["PDSunLon"][idx] - data["PDObsLon"][idx], "W")
print("")
print("Observer latitude", data["PDObsLat"][idx])
print("Sun latitude", data["PDSunLat"][idx])
print("Difference", data["PDSunLat"][idx] - data["PDObsLat"][idx])
print("")
print("Inclination of the map", data["inc"][idx])
print("xs", data["xs"][idx])
print("ys", data["ys"][idx])
print("zs", data["zs"][idx])

print(times.isot[idx])